In [6]:
import pyscf
from pyscf import tools, symm
import numpy as np

In [10]:
scf_energy=[]
molecule = """
    F   0.0 0.0 0.0
    F   2 0.0 0.0
    """
print(molecule)
#basis = "sto3g"
basis="6-31G"
mol = pyscf.gto.Mole(
    atom    =   molecule,
    symmetry=   True,
    spin    =   0,
    charge  =   0,
    basis   =   basis)


mol.build()
print("symmetry: ",mol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(mol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
scf_energy.append(mf.e_tot)
print(scf_energy)
norb = mf.mo_coeff.shape[0]
    #print(mf.mo_coeff)
print(norb)
pyscf.tools.molden.from_mo(mol, "C_RHF_f2.molden", mf.mo_coeff)
for s,i,c in zip(mol.irrep_name, mol.irrep_id, mol.symm_orb):
    print(" %5s %3i %4i"%(s, i, c.shape[1]))

def myocc(mf):    
    mol = mf.mol
    irrep_id = mol.irrep_id
    so = mol.symm_orb
    orbsym = symm.label_orb_symm(mol, irrep_id, so, mf.mo_coeff)
    occsym = np.array(orbsym)[mf.mo_occ==2]
    virsym = np.array(orbsym)[mf.mo_occ==0]
    for ir,irname in enumerate(mol.irrep_name):
        print('%5s, Occ = %5d, Vir = %d' % (irname, sum(occsym==ir), sum(virsym==ir)))
        print(orbsym)
        print(mol.irrep_id)
        print(mol.irrep_name)
myocc(mf)


    F   0.0 0.0 0.0
    F   2 0.0 0.0
    
symmetry:  Dooh


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 132 MB)
num. doubly occ = 9  num. singly occ = 0
init E= -198.051069905221
HOMO (E1gx) = -0.593017993660715  LUMO (A1u) = -0.511885793545614
cycle= 1 E= -198.51488412929  delta_E= -0.464  |g|= 0.212  |ddm|= 0.871
HOMO (A1g) = -0.601099690664782  LUMO (A1u) = -0.176454691046543
cycle= 2 E= -198.527944692556  delta_E= -0.0131  |g|= 0.061  |ddm|= 0.114
HOMO (A1g) = -0.572231786875531  LUMO (A1u) = -0.141329539184333
cycle= 3 E= -198.529025422701  delta_E= -0.00108  |g|= 0.0199  |ddm|= 0.0385
HOMO (A1g) = -0.577676059492466  LU

In [11]:
sym_map = {}
for ir,irname in enumerate(mol.irrep_id):
    sym_map[irname] = ir
    
clusters = []
for i in range(4):
    clusters.append([])
frozen= [0,1]  
orb_syms = {}
for i in range(norb):
        # print(" Orb: %5i %5s %5s " %(i, mf.orbsym[i], mf.orbsym[i]))
    print(" Orb: %5i %5s " %(i,  mol.irrep_name[sym_map[mf.orbsym[i]]]))
        # orb_syms.append(mol.irrep_name[sym_map[mf.orbsym[i]]])
    name = mol.irrep_name[sym_map[mf.orbsym[i]]]
    if i not in frozen:
        if name == "A1g" or name == "A1u":
            clusters[1].append(i)
        elif name == "E1gx" or name == "E1ux":
            clusters[2].append(i)
        elif name == "E1gy" or name == "E1uy":
            clusters[3].append(i)
if name in orb_syms:
    orb_syms[name].append(i)
else:
    orb_syms[name] = [i]    
            
    #specify frozen core
clusters[0] = [0,1]
C=mf.mo_coeff
C_ordered = np.zeros((norb,0))
for ci,c in enumerate(clusters):
    C_ordered = np.hstack((C_ordered, C[:,c]))

 Orb:     0   A1g 
 Orb:     1   A1u 
 Orb:     2   A1g 
 Orb:     3   A1u 
 Orb:     4  E1uy 
 Orb:     5  E1ux 
 Orb:     6  E1gx 
 Orb:     7  E1gy 
 Orb:     8   A1g 
 Orb:     9   A1u 
 Orb:    10  E1uy 
 Orb:    11  E1ux 
 Orb:    12   A1g 
 Orb:    13   A1u 
 Orb:    14  E1gx 
 Orb:    15  E1gy 
 Orb:    16   A1g 
 Orb:    17   A1u 


In [12]:
pyscf.tools.molden.from_mo(mol, "C_ordered_f2.molden", C_ordered)
init_fspace = [(2,2), (3,3), (1,1), (1,1)]
print(clusters)
print(init_fspace)
for ci in clusters:
    print(len(ci))
pyscf.scf.hf_symm.analyze(mf)
h0 = pyscf.gto.mole.energy_nuc(mol)
h1 = pyscf.scf.hf.get_hcore(mol)
h1 = C_ordered.T @ h1 @ C_ordered

nact = h1.shape[0]
h2 = pyscf.ao2mo.kernel(mol, C_ordered, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

S = mol.intor("int1e_ovlp_sph")

[[0, 1], [2, 3, 8, 9, 12, 13, 16, 17], [5, 6, 11, 14], [4, 7, 10, 15]]
[(2, 2), (3, 3), (1, 1), (1, 1)]
2
8
4
4
**** SCF Summaries ****
Total Energy =                        -198.529095028343704
Nuclear Repulsion Energy =              21.431677042260002
One-electron Energy =                 -320.735778080781301
Two-electron Energy =                  100.775006010177592
Wave-function symmetry = Dooh
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux
                               3    1    1    2    1    1
**** MO energy ****
MO #1 (A1g #1), energy= -26.4507867266572 occ= 2
MO #2 (A1u #1), energy= -26.4507267680701 occ= 2
MO #3 (A1g #2), energy= -1.62322524856184 occ= 2
MO #4 (A1u #2), energy= -1.56992436053027 occ= 2
MO #5 (E1uy #1), energy= -0.744717855670624 occ= 2
MO #6 (E1ux #1), energy= -0.744717855670627 occ= 2
MO #7 (E1gx #1), energy= -0.715578866866712 occ= 2
MO #8 (E1gy #1), energy= -0.715578866866709 occ= 2
MO #9 (A1g #3), energy= -0.577627774563499 occ= 2
MO #10 (A1u 

In [13]:
pyscf.scf.hf_symm.analyze(mf)

**** SCF Summaries ****
Total Energy =                        -198.529095028343704
Nuclear Repulsion Energy =              21.431677042260002
One-electron Energy =                 -320.735778080781301
Two-electron Energy =                  100.775006010177592
Wave-function symmetry = Dooh
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux
                               3    1    1    2    1    1
**** MO energy ****
MO #1 (A1g #1), energy= -26.4507867266572 occ= 2
MO #2 (A1u #1), energy= -26.4507267680701 occ= 2
MO #3 (A1g #2), energy= -1.62322524856184 occ= 2
MO #4 (A1u #2), energy= -1.56992436053027 occ= 2
MO #5 (E1uy #1), energy= -0.744717855670624 occ= 2
MO #6 (E1ux #1), energy= -0.744717855670627 occ= 2
MO #7 (E1gx #1), energy= -0.715578866866712 occ= 2
MO #8 (E1gy #1), energy= -0.715578866866709 occ= 2
MO #9 (A1g #3), energy= -0.577627774563499 occ= 2
MO #10 (A1u #3), energy= -0.146591891078355 occ= 0
MO #11 (E1uy #2), energy= 1.33795338201343 occ= 0
MO #12 (E1ux #2), energ

((array([1.99999852e+00, 1.99525012e+00, 3.04564653e-04, 1.00234239e+00,
         1.99916372e+00, 1.99916372e+00, 2.10440837e-03, 8.36279352e-04,
         8.36279352e-04, 1.99999852e+00, 1.99525012e+00, 3.04564653e-04,
         1.00234239e+00, 1.99916372e+00, 1.99916372e+00, 2.10440837e-03,
         8.36279352e-04, 8.36279352e-04]),
  array([ 3.55271368e-15, -1.42108547e-14])),
 array([-3.61203890e-14, -3.51011143e-48,  2.72174540e-31]))